Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

* **название источника;**

* **наименование новости;**

* **ссылку на новость;**

* **дата публикации.**

Сложить собранные новости в БД

In [1]:
import requests
from lxml import html
from pprint import pprint
from datetime import datetime

from pymongo import MongoClient

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client['my_database']
news = db.news

In [3]:
header = {'User-Agent': 
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
url = 'https://news.mail.ru/'

In [4]:
def get_mail_news():
    response = requests.get(url, headers = header)
    dom = html.fromstring(response.text)
    links = dom.xpath("//td[contains(@class, 'daynews__')]/div/a/@href")
    full_data = []
    for link in links:
        object_ = {}
        new_response = requests.get(link, headers = header)
        new_dom = html.fromstring(new_response.text)
        resourse = new_dom.xpath("//a[contains(@class, 'breadcrumbs__link')]/span/text()")
        name = new_dom.xpath("//h1[@class = 'hdr__inner']/text()")
        time = new_dom.xpath("//span[contains(@class, 'js-ago')]/@datetime")
        time = datetime.strptime(time[0], '%Y-%m-%dT%H:%M:%S%z')
        time = time.strftime("%d/%m/%Y %H:%M:%S")
        object_['link'] = link
        object_['title'] = name
        object_['resourse'] = resourse
        object_['time'] = time
        news.insert_one(object_)

In [5]:
get_mail_news()
for doc in news.find({}):
    pprint(doc)

{'_id': ObjectId('625721e2e01078ca3af0f77a'),
 'link': 'https://news.mail.ru/politics/50870541/',
 'resourse': ['© РИА Новости'],
 'time': '13/04/2022 15:53:41',
 'title': ['В Кремле прокомментировали возможность обмена Медведчука']}
{'_id': ObjectId('625721e3e01078ca3af0f77b'),
 'link': 'https://news.mail.ru/incident/50871266/',
 'resourse': ['Известия'],
 'time': '13/04/2022 16:50:05',
 'title': ['В Ростовской области задержали 26 сотрудников ГИБДД за взятки']}
{'_id': ObjectId('625721e3e01078ca3af0f77c'),
 'link': 'https://news.mail.ru/politics/50873195/',
 'resourse': ['© РИА Новости'],
 'time': '13/04/2022 18:51:54',
 'title': ['Вена, Берлин и Будапешт выступили против эмбарго на российский '
           'газ']}
{'_id': ObjectId('625721e3e01078ca3af0f77d'),
 'link': 'https://news.mail.ru/economics/50873962/',
 'resourse': ['Известия'],
 'time': '13/04/2022 20:15:19',
 'title': ['В России не ожидают дефицита ибупрофена из-за решения Reckitt '
           'передать бизнес']}
{'_id': O